### Import libraries

In [1]:
import os
import sys
import time
import pandas as pd
import geopandas as gpd
import arcpy

In [2]:
# show all columns
pd.options.display.max_columns = None

### Tables are stored in an Hierarchical Data Format version 5 (hdf5 or h5) 

In [3]:
# Read in hdf5 
store = pd.HDFStore('.\\Data\\remm_data_2015_base_year_02082019.h5')
tables = list(store.keys())
tables

['/buildings',
 '/buildings_for_estimation',
 '/buildings_for_estimation_grouped',
 '/employment_controls',
 '/household_controls',
 '/household_for_estimation',
 '/households',
 '/households_for_estimation',
 '/households_for_estimation1',
 '/jobs',
 '/parcels',
 '/travel_data',
 '/valid_parcels',
 '/zoning',
 '/zoning_base_line',
 '/zoning_baseline',
 '/zoning_for_parcels']

In [4]:
%%html
<style>
table {float:left}
</style>

#### Tasks
Update County ID in following tables:
- parcels
- jobs
- households
- household_controls
- employment_controls
- valid_parcels?? 


#### Files in h5 store that aren't being used by REMM
- household_controls (csv file is used instead)
- population_controls (csv file is used instead)
- employment_controls (csv file is used instead)


#### Identify parcels with 0 as county ID and identify their id (use TAZ)

**Crosswalk Table**

| CountyName| County_ID | CID  | FIPS |
|-----------|--------   |------|------|
| Box Elder |   5       | NA   |   3  |
| Davis     |   2       |  1   | 11   |
| Salt Lake |   3       |  2   |35   |
| Utah      |   4       |  3   |49   |
| Weber     |   1       |  4   |57   |  

  
  


In [5]:
# Create crosswalk dictionary for county values
crosswalk1 = {2:11, 3:35, 1:57, 4:49, 5:3, 0:0} # County_ID
crosswalk2 = {1:11, 2:35, 4:57, 3:49, 0:0} # CID

### Work on parcels

In [6]:
# get the parcels table from the store
parcels = store.get('parcels').reset_index()
parcels.head(5)

,parcel_id,Unnamed: 0,county_id,zone_id,parcel_acres,land_value,x,y,elevation,fwy_exit,airport,rail_depot,stream,trail,university,shape_area,volume_one_way,volume_two_way,airport_distance,fwy_exit_dist,raildepot_dist,university_dist,trail_dist,stream_dist,train_station,rail_stn_dist,bus_rte_dist,bus_stop,bus_stop_dist,volume_two_way_nofwy,distsml_id,distmed_id,distlrg_id,parent_parcel,CO_NAME,parcel_id_REMM,county_taz_id,utmxi,utmyi,city,zonal_ppa
0,741871,0,4,2111,0.197968,35700.00,1531239.186,7330722.151,1447.537450,0.0,0.0,0.0,1.0,0.0,0.0,8.623484e+03,992.0,1942.0,18.02655,0.85831,12.61234,14.73993,0.09764,0.0,0.0,8.748286,2382.31,0.0,0.882466,1942.0,487,38,13,741871,UTAH,741871,492111,424331.7633,4477427.732,Lehi,176926.527483
1,579822,1,4,2873,37.717834,56614.32,1535437.548,7341595.932,1657.284652,0.0,0.0,0.0,1.0,0.0,0.0,1.642989e+06,3564.0,7135.0,19.48828,1.32984,10.63629,15.94851,0.23207,0.0,0.0,6.691816,4930.50,0.0,1.164110,7135.0,0,0,0,579822,UTAH,579822,492873,425629.9856,4480733.739,Utah County,140869.334754
2,579853,2,4,2117,6.791235,81500.00,1534101.235,7334630.179,1657.284652,0.0,0.0,0.0,1.0,0.0,0.0,2.958262e+05,0.0,0.0,18.42054,1.66976,11.91035,15.00142,0.55081,0.0,0.0,7.994607,6770.95,0.0,1.661999,0.0,487,38,13,579853,UTAH,579853,492117,425210.6619,4478613.554,Lehi,199096.575130
3,640185,3,4,2073,1.372797,112100.00,1538253.186,7322787.258,1445.761763,0.0,0.0,0.0,1.0,0.0,0.0,5.979902e+04,2495.0,5043.0,16.06753,0.60111,14.23267,12.73670,0.04966,0.0,0.0,10.277766,2627.34,0.0,0.548186,5043.0,482,38,13,640185,UTAH,640185,492073,426455.1775,4474997.689,Lehi,169098.553773
4,682698,4,4,2119,0.163751,43000.00,1548648.398,7336198.043,1766.927829,0.0,0.0,0.0,1.0,0.0,0.0,7.132996e+03,3632.0,7133.0,17.61362,3.74880,12.20122,13.84603,0.37882,0.0,0.0,8.210217,9997.32,0.0,2.129948,7133.0,488,39,14,682698,UTAH,682698,492119,429646.0053,4479065.962,Draper,254583.603528


In [7]:
# Temporarily crosswalk values and get a value count
# crosswalk = {1:11, 2:35, 3:57, 4:49, 0:0}
parcels['county_id'].map(crosswalk1).value_counts()

11    368139
49    250210
57    113643
35    109672
0       3555
Name: county_id, dtype: int64

In [8]:
# Inspect a specific row
#parcels_copy[parcels_copy['CO_NAME']=='NO']['parcel_id_REMM'].tolist()
#parcels_copy[parcels_copy['parcel_id_REMM']==596194]

In [9]:
# parcels_unknown = parcels_copy[parcels_copy['county_id'] == 0].copy()
# parcels_unknown = parcels_unknown[['parcel_id_REMM', 'county_taz_id', 'CO_NAME', 'parcel_acres']]
# parcels_unknown.to_csv(os.path.join(r"./Results/", "parcels_unknown.csv"))

In [10]:
# # read in parcel shape
# parcels_shp = gpd.read_file('.\\Data\\Parcels_2019_UTM12.shp')

In [11]:
# # read in counties shape
# counties = gpd.read_file('.\\Data\\Counties_UTM12.shp')

In [12]:
# join =gpd.sjoin(parcels_shp, counties, op='intersects', how='left')
# #parcels_shp = parcel_shp[['parcel_id_', 'geometry']]
# # join parcels data to parcels shape

In [13]:
# read in parcels with county ids; 'parcel_id_' is 'parcel_id_REMM'; it was just cut odd
parcels_county = gpd.read_file('.\\Data\\parcels_county_id.shp')
parcels_county = parcels_county[['parcel_id_', 'FIPS']].copy()

In [14]:
print(parcels_county.shape)
print(parcels.shape)

(845219, 2)
(845219, 41)


In [15]:
# join resulting table back to original parcels table, keep new county fips column
parcels_converted = parcels.merge(parcels_county, left_on = 'parcel_id_REMM', right_on = 'parcel_id_' , how = 'inner')

# Store old cid for reference
parcels_converted['cid_old'] = parcels_converted['county_id']

# Replace old cid with FIPS codes
parcels_converted['county_id'] = parcels_converted['FIPS'].astype(int)

# Delete unneeded columns
del parcels_converted['FIPS']
del parcels_converted['parcel_id_']
del parcels_converted['Unnamed: 0']

# Check value counts
parcels_converted['county_id'].value_counts()

35    368588
49    253097
11    113748
57    109733
23        30
3         23
Name: county_id, dtype: int64

In [16]:
parcels_converted

,parcel_id,county_id,zone_id,parcel_acres,land_value,x,y,elevation,fwy_exit,airport,rail_depot,stream,trail,university,shape_area,volume_one_way,volume_two_way,airport_distance,fwy_exit_dist,raildepot_dist,university_dist,trail_dist,stream_dist,train_station,rail_stn_dist,bus_rte_dist,bus_stop,bus_stop_dist,volume_two_way_nofwy,distsml_id,distmed_id,distlrg_id,parent_parcel,CO_NAME,parcel_id_REMM,county_taz_id,utmxi,utmyi,city,zonal_ppa,cid_old
0,741871,49,2111,0.197968,35700.00,1531239.186,7330722.151,1447.537450,0.0,0.0,0.0,1.0,0.0,0.0,8.623484e+03,992.0,1942.0,18.02655,0.85831,12.61234,14.73993,0.09764,0.00000,0.0,8.748286,2382.31,0.0,0.882466,1942.0,487,38,13,741871,UTAH,741871,492111,424331.7633,4477427.732,Lehi,176926.527483,4
1,579822,49,2873,37.717834,56614.32,1535437.548,7341595.932,1657.284652,0.0,0.0,0.0,1.0,0.0,0.0,1.642989e+06,3564.0,7135.0,19.48828,1.32984,10.63629,15.94851,0.23207,0.00000,0.0,6.691816,4930.50,0.0,1.164110,7135.0,0,0,0,579822,UTAH,579822,492873,425629.9856,4480733.739,Utah County,140869.334754,4
2,579853,49,2117,6.791235,81500.00,1534101.235,7334630.179,1657.284652,0.0,0.0,0.0,1.0,0.0,0.0,2.958262e+05,0.0,0.0,18.42054,1.66976,11.91035,15.00142,0.55081,0.00000,0.0,7.994607,6770.95,0.0,1.661999,0.0,487,38,13,579853,UTAH,579853,492117,425210.6619,4478613.554,Lehi,199096.575130,4
3,640185,49,2073,1.372797,112100.00,1538253.186,7322787.258,1445.761763,0.0,0.0,0.0,1.0,0.0,0.0,5.979902e+04,2495.0,5043.0,16.06753,0.60111,14.23267,12.73670,0.04966,0.00000,0.0,10.277766,2627.34,0.0,0.548186,5043.0,482,38,13,640185,UTAH,640185,492073,426455.1775,4474997.689,Lehi,169098.553773,4
4,682698,49,2119,0.163751,43000.00,1548648.398,7336198.043,1766.927829,0.0,0.0,0.0,1.0,0.0,0.0,7.132996e+03,3632.0,7133.0,17.61362,3.74880,12.20122,13.84603,0.37882,0.00000,0.0,8.210217,9997.32,0.0,2.129948,7133.0,488,39,14,682698,UTAH,682698,492119,429646.0053,4479065.962,Draper,254583.603528,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845214,886695,49,2666,4.531522,48006.00,1574258.679,7220957.846,1372.488928,0.0,0.0,0.0,0.0,0.0,0.0,1.973931e+05,107.0,214.0,5.32442,4.38052,6.67391,8.88569,3.24828,0.01905,0.0,30.544293,19622.91,0.0,5.026164,214.0,546,44,16,601655,UTAH,886695,492666,437250.4157,4443906.735,Lake Shore,111196.498940,4
845215,886696,49,2666,3.158082,33456.00,1573684.703,7220955.159,1372.488928,0.0,0.0,0.0,0.0,0.0,0.0,1.375661e+05,107.0,214.0,5.32442,4.38052,6.67391,8.88569,3.24828,0.01905,0.0,30.544293,19622.91,0.0,5.026164,214.0,546,44,16,601655,UTAH,886696,492666,437075.5120,4443906.904,Lake Shore,111196.498940,4
845216,886697,49,2666,4.895412,51862.00,1573301.933,7220958.410,1372.488928,0.0,0.0,0.0,0.0,0.0,0.0,2.132442e+05,107.0,214.0,5.32442,4.38052,6.67391,8.88569,3.24828,0.01905,0.0,30.544293,19622.91,0.0,5.026164,214.0,546,44,16,601655,UTAH,886697,492666,436958.8819,4443908.552,Lake Shore,111196.498940,4
845217,886698,49,2666,4.918818,52109.00,1572835.592,7220958.477,1372.488928,0.0,0.0,0.0,0.0,0.0,0.0,2.142637e+05,107.0,214.0,5.32442,4.38052,6.67391,8.88569,3.24828,0.01905,0.0,30.544293,19622.91,0.0,5.026164,214.0,546,44,16,601655,UTAH,886698,492666,436816.7808,4443909.375,Lake Shore,111196.498940,4


### Work on households_for_estimation

In [17]:
households_for_estimation = pd.read_csv(".\\Data\\households_for_estimation.csv").set_index('household_id')
households_for_estimation.head()

,cars,household_type_id,persons,income,workers,children,tenure,recent_mover,building_type_id,block_id,building_id,proportion_workers,zone_id,parcel_id,node_id,income_quartile,county_id
household_id,,,,,,,,,,,,,,,,,
85,2,1,2,87500,2,0,2,1,1,17049,259924,1.000000,1074,152309,6119734,3,2
86,3,1,3,-1,3,0,2,0,1,6857,440101,1.000000,1607,347510,52501442,1,2
87,2,2,4,42500,2,2,2,0,1,13832,321465,0.500000,1095,224414,30362517,2,2
88,3,1,3,125000,2,0,2,0,1,4031,83948,0.666667,379,390199,6082196,4,2
89,2,2,5,87500,2,3,2,0,1,8950,221670,0.400000,1123,106020,6146478,3,1


In [18]:
# copy  the table
households_for_estimation_converted = households_for_estimation.copy()

# reclassify county values
households_for_estimation_converted['county_id'] = households_for_estimation_converted['county_id'].map(crosswalk1)

### Work on jobs

In [19]:
# get the jobs table from the store
jobs = store.get('jobs').reset_index()

# count instances of each county value
jobs['cid'].value_counts()

2    755026
4    272247
1    152689
3    115551
Name: cid, dtype: int64

In [20]:
# Create crosswalk dictionary for county values
# crosswalk = {2:11, 3:35, 1:57, 4:49, 0:0}

# copy  the table
jobs_converted = jobs.copy()

# reclassify county values
jobs_converted['cid'] = jobs['cid'].map(crosswalk2)

# count instances of each county value
jobs_converted['cid'].value_counts()

35    755026
57    272247
11    152689
49    115551
Name: cid, dtype: int64

### Work on households

In [21]:
# get the households table from the store
households = store.get('households').reset_index()

# count instances of each county value
households['cid'].value_counts()

2    377727
4    163909
1    106535
3     83221
Name: cid, dtype: int64

In [22]:
# Create crosswalk dictionary for county values
# crosswalk = {1:11, 2:35, 3:57, 4:49, 0:0}

# copy  the table
households_converted = households.copy()

# reclassify county values
households_converted['cid'] = households_converted['cid'].map(crosswalk2)

# count instances of each county value
households_converted['cid'].value_counts()

35    377727
57    163909
11    106535
49     83221
Name: cid, dtype: int64

### Work on household_controls

In [23]:
## get the household_controls table from the store
#household_controls2 = store.get('household_controls').reset_index()

# get the household_controls table from the csv
household_controls = pd.read_csv("./Data/household_controls.csv")

# count instances of each county value
household_controls['cid'].value_counts()

4    204
3    204
2    204
1    204
Name: cid, dtype: int64

In [24]:
# Create crosswalk dictionary for county values
# crosswalk = {1:11, 2:35, 3:57, 4:49, 0:0}

# copy  the table
household_controls_converted = household_controls.copy()

# reclassify county values
household_controls_converted['cid'] = household_controls_converted['cid'].map(crosswalk2)

# count instances of each county value
household_controls_converted['cid'].value_counts()

57    204
49    204
35    204
11    204
Name: cid, dtype: int64

In [25]:
# export the csv
household_controls_converted.to_csv(".\\Results\\household_controls.csv", index=False)

### Employment_Controls

In [26]:
# # get the jobs table from the store
# employment_controls = store.get('employment_controls').reset_index()

# get the household_controls table from the csv
employment_controls = pd.read_csv("./Data/employment_controls.csv")

# count instances of each county value
employment_controls['cid'].value_counts()

3    612
1    612
4    612
2    612
Name: cid, dtype: int64

In [27]:
# Create crosswalk dictionary for county values
# crosswalk = {1:11, 2:35, 3:57, 4:49, 0:0}

# copy  the table
employment_controls_converted = employment_controls.copy()

# reclassify county values
employment_controls_converted['cid'] = employment_controls_converted['cid'].map(crosswalk2)

employment_controls_converted['cid'].value_counts()

57    612
49    612
35    612
11    612
Name: cid, dtype: int64

In [28]:
# export the csv
employment_controls_converted.to_csv(".\\Results\\employment_controls.csv", index=False)

### Population_Controls

In [29]:
pop_controls = pd.read_csv("./Data/population_controls.csv")
pop_controls.head()

,year,number_of_population,cid
0,2010,304323,1
1,2010,1013395,2
2,2010,222446,3
3,2010,503812,4
4,2011,309937,1


In [30]:
# count instances of each county value
pop_controls['cid'].value_counts()

4    51
3    51
2    51
1    51
Name: cid, dtype: int64

In [31]:
# Create crosswalk dictionary for county values
# crosswalk = {1:11, 2:35, 3:57, 4:49, 0:0}

# copy  the table
pop_controls_converted = pop_controls.copy()

# reclassify county values
pop_controls_converted['cid'] = pop_controls_converted['cid'].map(crosswalk2)

pop_controls_converted['cid'].value_counts()

57    51
49    51
35    51
11    51
Name: cid, dtype: int64

In [32]:
# export the csv
pop_controls_converted.to_csv(".\\Results\\population_controls.csv", index=False)

### tazctyeq

In [33]:
tazctyeq = pd.read_csv(r"E:\Scripts\Jupyter_Practice\County_ID_Update\Data\TAZCTYEQ.csv")

# copy  the table
tazctyeq_converted = tazctyeq.copy()

# Create crosswalk dictionary for county values
# crosswalk = {1:11, 2:35, 3:57, 4:49, 5:3, 0:0}

# reclassify county values
tazctyeq_converted['COUNTY'] = tazctyeq['COUNTY'].map(crosswalk1)

# export the csv
tazctyeq_converted.to_csv(".\\Results\\TAZCTYEQ.csv", index=False)

In [34]:
tazctyeq_converted['COUNTY'].value_counts()

35    1134
49    1085
57     283
11     231
3      140
Name: COUNTY, dtype: int64

#### Load tables into h5

In [35]:
# store path for new hdf
new_hdf = '.\\Results\\remm_data_2015_base_year_09282020.h5'

# if it exists already delete it; it will not overwrite
if os.path.exists(new_hdf):
    try:
        new_hdf.close()
    except:
        pass
    
    os.remove(new_hdf)

# Create empty h5   
hdf = pd.HDFStore(new_hdf)   

In [36]:
# load the new tables to the h5
hdf.put('parcels', parcels_converted.set_index('parcel_id'), format='t', data_columns=True)
hdf.put('jobs', jobs_converted.set_index('jobs_id'), format='t', data_columns=True)
hdf.put('households', households_converted, format='t', data_columns=True)
# hdf.put('household_controls', household_controls_converted.set_index('year'), format='t', data_columns=True)
# hdf.put('employment_controls', employment_controls_converted.set_index('year'), format='t', data_columns=True)

In [37]:
# load the unchanged tables to the h5
hdf.put('buildings_for_estimation', pd.read_csv(".\\Data\\buildings_for_estimation.csv").set_index('building_id'), format='t', data_columns=True)

In [38]:
hdf.put('buildings_for_estimation_grouped', pd.read_csv(".\\Data\\buildings_for_estimation_grouped.csv").set_index('building_id'), format='t', data_columns=True)

In [39]:
hdf.put('buildings', pd.read_csv(".\\Data\\buildings.csv").set_index('building_id'), format='t', data_columns=True)

In [40]:
# hdf.put('household_for_estimation', pd.read_csv(".\\Data\\household_for_estimation.csv").set_index('household_id'), format='t', data_columns=True) # Not used

In [41]:
hdf.put('households_for_estimation', households_for_estimation_converted, format='t', data_columns=True)

In [42]:
# hdf.put('households_for_estimation1', pd.read_csv(".\\Data\\households_for_estimation1.csv").set_index('household_id'), format='t', data_columns=True) # Not used

In [43]:
# hdf.put('travel_data', pd.read_csv(".\\Data\\travel_data.csv").set_index(['from_zone_id', 'to_zone_id']), format='t', data_columns=True)

In [44]:
# hdf.put('valid_parcels', pd.read_csv(".\\Data\\valid_parcels.csv").set_index('parcel_id'), format='t', data_columns=True) # used in datasources.py; commented out

In [45]:
hdf.put('zoning', pd.read_csv(".\\Data\\zoning.csv").set_index('id'), format='t', data_columns=True)

In [46]:
# hdf.put('zoning_base_line', pd.read_csv(".\\Data\\zoning_base_line.csv").set_index('parcel_id'), format='t', data_columns=True) # Not used

In [47]:
hdf.put('zoning_baseline', pd.read_csv(".\\Data\\zoning_baseline.csv").set_index('parcel_id'), format='t', data_columns=True)

In [48]:
hdf.put('zoning_for_parcels', pd.read_csv(".\\Data\\zoning_for_parcels.csv").set_index('parcel'), format='t', data_columns=True)

In [49]:
hdf.close()